## Launch the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [1]:
import os
import shutil
import numpy as np
from pathlib import Path
import subprocess
import datetime
import time

#### Create an executable file of STEMMUS_SCOPE

In [2]:
# path to executable
root_path_to_model = Path(Path.home(), "STEMMUS_SCOPE")
# submit slurm job if the executable does not exist
if not Path(root_path_to_model, "exe", "STEMMUS_SCOPE").is_file():
    result = subprocess.run(["sbatch", "build_stemmus_scope_exe.sh"], cwd = Path(root_path_to_model, "exe"))
    result.check_returncode()

#### Update/set config files

In [3]:
# the user must provide the correct path
path_dict = {
    "SoilPropertyPath": "/projects/0/einf2480/model_parameters/soil_property/",
    "InputPath": "/scratch-shared/ecoextreml/stemmus_scope/test/input/",
    "OutputPath": "/scratch-shared/ecoextreml/stemmus_scope/test/output/",
    "ForcingPath": "/projects/0/einf2480/forcing/plumber2_data/",
    "ForcingFileName": "FI-Hyy_1996-2014_FLUXNET2015_Met.nc",
    "VegetationPropertyPath": "/projects/0/einf2480/model_parameters/vegetation_property/",
    "DurationSize": "20"
}

# generate a text file
root_path = Path().resolve().parent

with open(Path(root_path, "config_file_snellius.txt"), 'w') as f:
    print("The listed paths are defined in the config file:")
    for i in path_dict.keys():
        print(i)
        f.write((i + "=" + path_dict[i] + "\n"))

The listed paths are defined in the config file:
SoilPropertyPath
InputPath
OutputPath
ForcingPath
ForcingFileName
VegetationPropertyPath
DurationSize


#### Create input directories, prepare input files 

In [4]:
def input_dir(station_name):
    """Create input directory and prepare input files
    """
    # get start time with the format Y-M-D-HM
    start_time = str(datetime.datetime.now()).split('.')[0]
    timestamp = start_time.replace(' ', '-').replace(':', '')[:-2]
    # create input directory
    work_dir = Path(path_dict["InputPath"], station_name + '_' + timestamp)
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    print(f"Prepare work directory for the station: {station_name}")
    # copy model parameters to work directory
    shutil.copytree(path_dict["VegetationPropertyPath"], work_dir, dirs_exist_ok=True)
    # update config file for ForcingFileName and InputPath
    with open(Path(work_dir, f"{station_name}_{timestamp}_config.txt"), 'w') as f:
        for i in path_dict.keys():
            if i == "ForcingFileName":
                f.write(i + "=" + ncfile.stem + ".nc" + "\n")
            elif i == "InputPath":
                f.write(i + "=" + str(work_dir) + "/" + "\n")
            else:
                f.write(i + "=" + path_dict[i] + "\n")
    
    # create an output file
    Path(root_path, "out").mkdir(parents=True, exist_ok=True)
    with open(Path(root_path, "out", f"{station_name}.out"), 'w') as f:
        for i in path_dict.keys():
            if i == "ForcingFileName":
                f.write(i + " is " + ncfile.stem + ".nc" + "\n")
            elif i == "InputPath":
                f.write(i + " is " + str(work_dir) + "/" + "\n")
            elif i == "DurationSize":
                pass
            else:
                f.write(i + " is " + path_dict[i] + "\n")
    

In [5]:
# specify the station of interest
# station_names_list is by default none, if it is not specified, then all stations with forcing
# listed in the ForcingPath will be used.
#station_names_list = None
station_names_list = ["NL-Hor", "ZA-Kru"]

for ncfile in Path(path_dict["ForcingPath"]).iterdir():
    # get the station name
    station_name = ncfile.stem.split('_')[0]
    # check if a station list is provided
    if station_names_list == None:
        input_dir(station_name)
    elif station_name in station_names_list:
        input_dir(station_name)

Prepare work directory for the station: NL-Hor
Prepare work directory for the station: ZA-Kru


#### Run the model

In [6]:
for station_dir in os.listdir(path_dict["InputPath"]):
    station_name = station_dir.split("_")[0]
    path_to_config = Path(path_dict["InputPath"], f"{station_dir}",f"{station_dir}_config.txt")
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    os.environ['MATLAB_LOG_DIR'] = f'{Path(root_path, "out")}'
    # start time
    start_time = time.time()
    # run the model
    print(f"Run STEMMUS-SCOPE for the station: {station_name}")
    result = subprocess.run([f"exe/STEMMUS_SCOPE {path_to_config}", ">>", f"{station_name}.out"], cwd = root_path_to_model, shell=True)
    result.check_returncode()
    # calculate total execution time
    run_time = time.time() - start_time
    # add execution information to out file
    with open(Path(root_path, "out", f"{station_name}.out"), 'a') as f:
        f.write(f"Run is COMPLETED. Model run time is {run_time} s.")

Opening log file:  /gpfs/home3/yangl/processing/out/java.log.8605
Reading config from /scratch-shared/ecoextreml/stemmus_scope/test/input/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_config.txt

 The calculations start now 
Opening log file:  /gpfs/home3/yangl/processing/out/java.log.9641
Reading config from /scratch-shared/ecoextreml/stemmus_scope/test/input/NL-Hor_2022-05-10-1135/NL-Hor_2022-05-10-1135_config.txt

 The calculations start now 


#### Create output directories, prepare output files

In [7]:
# convert csv files to nc files
path_to_utils = Path(root_path_to_model, "utils/csv_to_nc")
for station_dir in os.listdir(path_dict["InputPath"]):
    path_to_config = Path(path_dict["InputPath"], f"{station_dir}",f"{station_dir}_config.txt")
    result = subprocess.run(["python", Path(path_to_utils, "generate_netcdf_files.py"),
         "--config_file", path_to_config, "--variable_file", Path(path_to_utils, "Variables_will_be_in_NetCDF_file.csv")])
    result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_STEMMUS_SCOPE.nc
/scratch-shared/ecoextreml/stemmus_scope/test/output/NL-Hor_2022-05-10-1136/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/test/output/NL-Hor_2022-05-10-1136/NL-Hor_202